In [1]:
from IPython.display import display
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
from scipy.stats import zscore
import gzip
import os
import csv
from alive_progress import alive_bar
from collections import defaultdict


years_Columns = list(range(1800,2000)) 
directory = 'C:/Users/bincl/BA-Thesis/Dataset/2gram/'


c:\Users\bincl\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, element_at, slice, size, regexp_extract, transform, when, explode, \
monotonically_increasing_id, map_from_arrays, lit, udf,collect_list, row_number, ceil, map_keys, expr, from_json
from pyspark.sql.types import ShortType, ArrayType, LongType, StringType
from pyspark.sql import types as T

import pandas as pd

#path = 'C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_ab.gz'
path = 'C:/Users/bincl/BA-Thesis/Dataset/3gram/3_20000_nopos_sample/3_20000_nopos_sample.gz'

spark = SparkSession.builder.appName('3gramSQL').getOrCreate()

raw_input_df = spark \
                .read.csv(path, sep='\n',quote="").withColumnRenamed('_c0', 'Input')

split_df = raw_input_df \
                .select(split('Input', '\t').alias('SplitInput')) \
                .select(element_at('SplitInput', 1).alias('Tokens'),
                        slice('SplitInput', 2,
size('SplitInput')).alias('Data')) \
                .select('Tokens', 'Data') \

df_3gram = split_df.select('Tokens', transform('Data', lambda d:
split(d, ',')).alias('Data')) \
                .select('Tokens', transform('Data', lambda x:
x[0]).alias('Year'),
                        transform('Data', lambda x:
x[1]).cast(ArrayType(LongType())).alias('Occurrences')) \
                .select('Tokens', map_from_arrays('Year',
'Occurrences').alias('Data')) \
                .select(['Tokens', 'Data'])              

In [3]:
from os import listdir
from os.path import isfile, join

pathlist = [directory + f for f in listdir(directory) if isfile(join(directory, f))]

spark2 = SparkSession.builder.appName('3gramSQL').getOrCreate()

raw_input_df2 = spark2 \
                .read.csv(pathlist, sep='\n',quote="").withColumnRenamed('_c0', 'Input')

split_df2 = raw_input_df2 \
                .select(split('Input', '\t').alias('SplitInput')) \
                .select(element_at('SplitInput', 1).alias('Tokens'),
                        slice('SplitInput', 2,
size('SplitInput')).alias('Data')) \
                .select('Tokens', 'Data') \

df_2gram = split_df2.select('Tokens', transform('Data', lambda d:
split(d, ',')).alias('Data')) \
                .select('Tokens', transform('Data', lambda x:
x[0]).alias('Year'),
                        transform('Data', lambda x:
x[1]).cast(ArrayType(LongType())).alias('Occurrences')) \
                .select('Tokens', map_from_arrays('Year',
'Occurrences').alias('Data')).select(['Tokens', 'Data'])
            

In [4]:
# if id set as default parameter then a list is given if not a spark dataframe is given as parameter df
def get_pd_df(df):
    
    matched = df['Data'] 
    matched_pandas_df = pd.Series(matched)
    matched_pandas_df.sort_index(inplace = True)

    matched_pandas_df.index.astype('int')
    numbers = list(range(1500,2020))
    numbers = map(str, numbers)
    matched_pandas_df = matched_pandas_df.reindex(numbers, fill_value= 0)
    return matched_pandas_df

def get_pd_df_from_string(suchstring, df):
    x = df.select(['Tokens', 'Data']).where(df.Tokens == suchstring)
    matched = x.first()['Data']
    matched_pandas_df = pd.Series(matched)
    matched_pandas_df.sort_index(inplace = True)

    matched_pandas_df.index.astype('int')
    numbers = list(range(1500,2020))
    numbers = map(str, numbers)
    matched_pandas_df = matched_pandas_df.reindex(numbers, fill_value= 0)
    return matched_pandas_df

In [5]:
# deprecated zu langsam
def find_n_gram(direc, find):
    for filename in os.listdir(direc):
        f = os.path.join(direc, filename)
        # checking if it is a file
        if os.path.isfile(f):
            print(f)
            with gzip.open(f,'rt', encoding='utf-8') as input:
                str_arr_csv = input.readlines()
                for line in str_arr_csv:
                    token = line.split("\t")
                    if find == token[0]:
                        return line
                        
    return "line not found" 

def start_the_search(directory,search):
    result = find_n_gram(directory,search)
    
    if result == "line not found":
        print(result)
        exit()
    result = result.split("\t")
    lower_ngram = pd.Series(result[1:],dtype= "string", name = result[0])
    return lower_ngram

In [6]:
def getLowerLevelString(string):
   first, *middle, last = string.split()
   first = first + " " + ''.join(middle)
   last = ''.join(middle) + " " + last
   return first, last

def MLR(data):
    token = data['Tokens']
    first,last = getLowerLevelString(token)
    print(first)
    print(last)
    first = get_pd_df_from_string(first,df_2gram)
    last = get_pd_df_from_string(last,df_2gram)
    
    X = pd.concat([first,last],axis=1)
    y = get_pd_df(data)
    print(X)
    print(y)
    reg = LinearRegression()
    reg.fit(X, y) 
    
    return reg.coef_,reg.intercept_,X,y

def buildApproximation(c1,c2,basevalue,df):
    df = df.astype('float')
    df['scaledFirst'] = df.iloc[:,0].apply(lambda x: x * c1) 
    df['scaledLast'] = df.iloc[:,1].apply(lambda x: x * c2) 
    df['approximation'] = df['scaledFirst'] + df['scaledLast'] + basevalue
    return df

In [7]:
def compressWithError2gram(firstN,error):
    df_3_gram = df_3gram.head(firstN)
    result = []
    all = []
    sum = []
    
    
    with alive_bar(firstN) as bar:
        
        for i in range(1,firstN):
            data = df_3_gram[i]
            token = data['Tokens']
            coef,intercept,dfAprox,original = MLR(data)
            c1,c2 = coef
            df = buildApproximation(c1,c2,intercept,dfAprox)
            dfOriginal = pd.DataFrame()
            dfOriginal['values'] = original
            dfOriginal['values'] = dfOriginal['values'].apply(pd.to_numeric, errors='coerce')
            #dfOriginal= pd.to_numeric(dfOriginal)
            dfOriginal['zscore'] = zscore(dfOriginal)
            df['zscore'] = zscore(df['approximation'])
            
            sum.append(pd.to_numeric(dfOriginal['values']).sum()) 
            
            rmse = mean_squared_error(dfOriginal['zscore'], df['zscore'], squared = False)
            #rmse = mean_squared_error(dfOriginal, df['approximation'])
            if rmse <= error:
                result.append([token,rmse,dfOriginal['values'],dfOriginal['zscore'],df['approximation'],df['zscore']]) 
            all.append(rmse)    
            bar()
    return result, all, sum


In [8]:
result, all, sum = compressWithError2gram(10, 0.05)
print(len(result))



on 0: [Row(Tokens='" Bis seine', Data={'2012': 6, '2011': 1, '2010': 3, '1998': 10, '1876': 1, '1754': 1, '1875': 2, '1897': 2, '1952': 2, '1852': 2, '1951': 1, '1993': 1, '2003': 2, '1936': 1, '1736': 1, '2001': 1, '2000': 1, '2019': 1, '1985': 1, '2018': 1, '1841': 1, '2017': 10, '1840': 2, '1884': 1, '1961': 1, '2016': 5, '2015': 5, '2014': 5, '2013': 7, '1828': 1, '1927': 4, '1867': 1, '1922': 10}), Row(Tokens='" -- Vorgehen', Data={'2012': 1, '2011': 4, '2010': 5, '1998': 5, '2009': 1, '1975': 2, '1997': 3, '2008': 13, '1996': 4, '2007': 6, '1973': 1, '2006': 1, '1972': 1, '2005': 7, '1971': 1, '1993': 1, '2004': 3, '1992': 1, '2003': 4, '1991': 1, '2002': 3, '1916': 1, '1979': 1, '1978': 3, '1977': 2, '1999': 4, '1909': 1, '1908': 1, '1990': 2, '2001': 3, '2000': 8, '1986': 1, '2019': 4, '1886': 2, '2018': 3, '1984': 4, '2017': 4, '1983': 2, '2016': 3, '1982': 1, '2015': 3, '2014': 5, '1980': 2, '2013': 28, '1907': 1, '1902': 2, '1989': 2, '1988': 1}), Row(Tokens='" : Oesterreich

TypeError: 'NoneType' object is not subscriptable

In [ ]:
plt.boxplot(all)
plt.show()
rmse_with_error = []

for i in result:
    rmse_with_error.append(i[1])
plt.boxplot(rmse_with_error)
plt.show()

import seaborn as sns

sns.violinplot(x= rmse_with_error, inner="point")

In [ ]:
sns.violinplot(x= all, inner_kws=dict(box_width=15, whis_width=2, color=".8"))

In [ ]:

plt.scatter(sum, all)
plt.xscale('log')
plt.show()